In [38]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable
import pandas as pd
import math
import csv
import numpy as np

df = pd.read_csv('../../Data/SMTO_2019/SMTO_2019_Complete_Input.csv')

df['Family'].replace('', np.nan, inplace=True)
df.dropna(subset = ['Family'], inplace=True)

#df = df[df['Level'] != 'Other']
school_codes = df['School'].unique().tolist()
df['School'] = df['School'].apply(lambda x: school_codes.index(x))

uni_df = df[df['School_Type'] == 'University']
col_df = df[df['School_Type'] == 'College']

full_df = df.copy()
df.head()

,Liv_Arr,Children,Cars,Income,Home_Zone,School_Name,Campus,Work,Licence,Mode,...,Dist.SHD,Dist.SHH,Dist.SHT,Dist.MI,Dist.SC,Dist.SG,Dist.YK,Dist.YG,Dist.RY,Dist.OC
0,Live with family/parents,0.0,0.0,NaN,3851.0,Centennial College,Progress Campus,NW,False,Transit,...,29.60268,4.821538,16.18253,19.03615,54.90925,32.749820,40.58064,43.982800,33.22763,31.52583
1,Live with family/parents,0.0,1.0,NaN,181.0,Centennial College,Morningside Campus,NW,False,Transit,...,35.02901,43.017220,51.23784,22.93553,22.04248,8.990107,9.21471,8.279897,10.30155,10.30233
2,Live with family/parents,0.0,2.0,I don't know,1039.0,Centennial College,Progress Campus,NW,False,Transit,...,68.36338,71.352090,81.92638,54.53982,13.64408,35.730980,40.61661,29.418650,35.05555,36.44957
4,Live with family/parents,1.0,1.0,NaN,600.0,Centennial College,Progress Campus,NW,False,NaN,...,49.27058,61.537790,69.75841,41.28113,11.49956,25.838390,18.81172,15.841280,25.16296,26.82406
5,Live with roommates,NaN,0.0,I don't know,544.0,Centennial College,Progress Campus,PT,False,Transit,...,50.58400,48.825410,59.43179,32.18690,10.84569,13.236390,27.52188,12.418690,12.52769,13.66335


In [39]:
def run_model(name, with_ASCs, combined = True):
    database = db.Database("SMTO", df.select_dtypes(include = 'number'))
    ASCs, V, av = [], {}, {}
    B_DIST = Beta('B_DIST', 0, None, None, 0)
    #B_ENROL = Beta('B_ENROL', 1, None, None, 1 if with_ASCs else 0)
    B_FAM_DIST = Beta('B_FAM_DIST', 0, None, None, 0)

    for i in range(len(school_codes)):
        code = school_codes[i]
        ASCs.append(Beta('ASC_' + code, 0, None, None, 0 if with_ASCs and code != 'SG' else 1))
        if combined:
            V[i] = ASCs[i] + B_DIST * database.variables['Dist.' + code] + B_FAM_DIST * database.variables["Dist." + code] * database.variables["Family"]
        else:
            V[i] = ASCs[i] + B_DIST * database.variables['Dist.' + code]
        av[i] = 1   
        
    logprob = models.loglogit(V, av, database.variables["School"])
    #test_dict = {'loglike': logprob, 'weight': database.variables["Exp_Segment"]}
    biogeme  = bio.BIOGEME(database,test_dict,numberOfThreads=1)
    biogeme.modelName = name
    results = biogeme.estimate(saveIterations=True)
    betas = results.getBetaValues()
    
    print("Results for " + name + " model:")
    print(betas)
    print()
    return betas

In [40]:
def get_cm(name, betas, with_ASCs, combined = True):
    
    for i in range(len(school_codes)):
        code = school_codes[i]
        if with_ASCs:
            if combined:
                df['V_'+ code] = (betas['ASC_' + code] if code != 'SG' else 0) + betas['B_DIST'] * df['Dist.' + code] + betas['B_FAM_DIST'] * df['Dist.' + code] * df['Family']
            else:
                df['V_'+ code] = (betas['ASC_' + code] if code != 'SG' else 0) + betas['B_DIST'] * df['Dist.' + code]
        else:
            if combined:
                df['V_'+ code] = betas['B_DIST'] * df['Dist.' + code] + betas['B_FAM_DIST'] * df['Dist.' + code] * df['Family']
            else:
                df['V_'+ code] = betas['B_DIST'] * df['Dist.' + code]

    utils = df.iloc[:,-7:]
    for i in range(len(school_codes)):
        code = school_codes[i]
        df['P_' + code] = utils.apply(lambda x: math.exp(x['V_' + code]) / sum([math.exp(j) for j in x]), axis = 1)
    probs = pd.concat((df['School'], df.iloc[:,-7:]), axis=1)
    
    print("Softmax confusion matrix for " + name + " model:")
    softmax_cm = []
    for school in range(len(school_codes)):
        softmax_cm.append(probs[probs['School'] == school][['P_' + i for i in school_codes]].sum().values.tolist())
        print(*probs[probs['School'] == school][['P_' + i for i in school_codes]].sum().values)
    
    print(softmax_cm)
    accuracy = 0
    total = 0
    for i in range(7):
        total += sum(softmax_cm[i])
        accuracy += softmax_cm[i][i]
    accuracy = accuracy/total
    print('\nAccuracy: ' + str(accuracy))
    
    
    print("\nHardmax confusion matrix for " + name + " model:")
    for school in range(len(school_codes)):
        print(*[(probs[probs['School'] == school][['P_' + i for i in school_codes]].idxmax(axis = 1) == 'P_' + j).sum() for j in school_codes])
    
    print()

In [30]:
'''
for (x, y) in (('Eric', True),('Proposed', False)):
    df = full_df.copy()
    print("----------- Combined ----------")
    get_cm(x, run_model(x, y), y)
    
    #print("----------- Family ----------")
    #df = full_df[full_df['Family'] == 1]
    #get_cm(x, run_model(x, y, False), y, False)
    
    #print("----------- Non-Family ----------")
    #df = full_df[full_df['Family'] == 0]
    #get_cm(x, run_model(x, y, False), y, False)
'''

'\nfor (x, y) in ((\'Eric\', True),(\'Proposed\', False)):\n    df = full_df.copy()\n    print("----------- Combined ----------")\n    get_cm(x, run_model(x, y), y)\n    \n    #print("----------- Family ----------")\n    #df = full_df[full_df[\'Family\'] == 1]\n    #get_cm(x, run_model(x, y, False), y, False)\n    \n    #print("----------- Non-Family ----------")\n    #df = full_df[full_df[\'Family\'] == 0]\n    #get_cm(x, run_model(x, y, False), y, False)\n'

In [41]:
df = full_df.copy()
print("----------- Combined ----------")
get_cm('Proposed', run_model('Proposed', True), True)

----------- Combined ----------


biogemeError: The database contains NaN value(s). Detect where they are using the function isnan()